In [1]:
!hostnamectl

 Static hostname: DGX
       Icon name: computer-server
         Chassis: server 🖳
      Machine ID: dedf5691225742ff95b33f842b0781cc
         Boot ID: cb04df0ff1774eeda112c63867eeefdd
Operating System: ]8;;https://www.ubuntu.com/Ubuntu 24.04.2 LTS]8;;              
          Kernel: Linux 6.8.0-53-generic
    Architecture: x86-64
 Hardware Vendor: NVIDIA
  Hardware Model: DGXA100 920-23687-2530-002
Firmware Version: 1.29
   Firmware Date: Tue 2024-08-06
    Firmware Age: 7month 2w 5d


In [2]:
import torch
import pandas as pd

In [3]:
torch.cuda.is_available()

True

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fedesoriano/cifar100")

print("Path to dataset files:", path)

Path to dataset files: /home/ir739wb/.cache/kagglehub/datasets/fedesoriano/cifar100/versions/1


In [6]:
%ls /home/ir739wb/.cache/kagglehub/datasets/fedesoriano/cifar100/versions/1

file.txt  meta  test  train


In [7]:
%pwd

'/home/ir739wb/ilyarekun/nn-lessons'

In [8]:
%cd

/home/ir739wb


In [9]:
%ls

ilyarekun/  miniconda3/


In [10]:
%cd ilyarekun/nn_DeiT_25/

/home/ir739wb/ilyarekun/nn_DeiT_25


In [11]:
%ls

README.md  error.log  jbdeit.slurm  output.log
data/      fbkrep/    notebooks/    todo.md


In [14]:
%cp -r /home/ir739wb/.cache/kagglehub/datasets/fedesoriano/cifar100/versions/1 ./data

In [3]:
%pwd

'/home/ir739wb/ilyarekun/nn-lessons'

In [4]:
%cd /home/ir739wb/ilyarekun/nn_DeiT_25/

/home/ir739wb/ilyarekun/nn_DeiT_25


In [5]:
import os
import numpy as np
import pickle
from PIL import Image
from sklearn.model_selection import train_test_split

# Функция для загрузки данных из файла CIFAR-100
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Путь к скачанному датасету
path = "/home/ir739wb/.cache/kagglehub/datasets/fedesoriano/cifar100/versions/1"  # Укажи свой путь
data_pre_path = "./data/1"  # Куда ты скопировал данные

# Создание директорий
output_dir = "./cifar100_imagenet_style"
train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "val")
test_dir = os.path.join(output_dir, "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Загрузка метаданных для названий классов
metadata_path = os.path.join(data_pre_path, "meta")
metadata = unpickle(metadata_path)
fine_label_names = [name.decode('utf-8') for name in metadata[b'fine_label_names']]  # 100 классов

# Создание папок для каждого класса в train, val и test
for label_name in fine_label_names:
    os.makedirs(os.path.join(train_dir, label_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, label_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, label_name), exist_ok=True)

# Загрузка тренировочных данных
data_train_path = os.path.join(data_pre_path, "train")
data_train_dict = unpickle(data_train_path)
data_train = data_train_dict[b'data']  # (50000, 3072)
label_train = np.array(data_train_dict[b'fine_labels'])  # (50000,)

# Загрузка тестовых данных
data_test_path = os.path.join(data_pre_path, "test")
data_test_dict = unpickle(data_test_path)
data_test = data_test_dict[b'data']  # (10000, 3072)
label_test = np.array(data_test_dict[b'fine_labels'])  # (10000,)

# Объединение данных
data_all = np.vstack((data_train, data_test))  # (60000, 3072)
label_all = np.hstack((label_train, label_test))  # (60000,)

# Разделение на train (70%), val (15%), test (15%) с учетом классов
train_idx, temp_idx, train_labels, temp_labels = train_test_split(
    np.arange(len(label_all)), label_all, test_size=0.3, stratify=label_all, random_state=42
)
val_idx, test_idx, val_labels, test_labels = train_test_split(
    temp_idx, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42
)

# Функция для сохранения изображения
def save_image(data, label, index, folder):
    img_data = data[index].reshape(3, 32, 32).transpose(1, 2, 0)  # Преобразование из (3072,) в (32, 32, 3)
    img = Image.fromarray(img_data)
    class_name = fine_label_names[label[index]]
    img_path = os.path.join(folder, class_name, f"img{index}.jpeg")
    img.save(img_path)

# Сохранение изображений в train, val, test
for idx in train_idx:
    save_image(data_all, label_all, idx, train_dir)

for idx in val_idx:
    save_image(data_all, label_all, idx, val_dir)

for idx in test_idx:
    save_image(data_all, label_all, idx, test_dir)

print("Данные успешно разделены на train (70%), val (15%), test (15%) и сохранены!")

Данные успешно разделены на train (70%), val (15%), test (15%) и сохранены!


In [7]:
import os

# Путь к основной директории
base_path = "/home/ir739wb/ilyarekun/nn_DeiT_25/cifar100_imagenet_style"

# Функция для подсчёта файлов в подпапке (train или val)
def count_files_in_subfolder(subfolder):
    subfolder_path = os.path.join(base_path, subfolder)
    total_files = 0
    for root, dirs, files in os.walk(subfolder_path):
        total_files += len(files)
    return total_files

# Подсчёт файлов в train и val
train_files = count_files_in_subfolder("train")
val_files = count_files_in_subfolder("val")
test_files = count_files_in_subfolder("test")

# Вывод результатов
print(f"Количество файлов в train: {train_files}")
print(f"Количество файлов в val: {val_files}")
print(f"Количество файлов в val: {test_files}")

Количество файлов в train: 42000
Количество файлов в val: 9000
Количество файлов в val: 9000
